# Чтение данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
import warnings
import warnings
warnings.filterwarnings('ignore', category=Warning)

In [4]:
tags = pd.read_csv('data/tags.csv', sep=',#', names=['tag', 'name', 'type'])
tags = tags.loc[tags['type'] == '"1"']
tags.head(10)

,tag,name,type
151,"""179""","""О войне""","""1"""
152,"""180""","""военный""","""1"""
153,"""181""","""заграничная поездка""","""1"""
154,"""182""","""блокада""","""1"""
155,"""183""","""тыл""","""1"""
156,"""184""","""фронтовой""","""1"""
157,"""185""","""депортация""","""1"""
158,"""186""","""детский""","""1"""
159,"""187""","""профессиональный""","""1"""
160,"""188""","""путешествие""","""1"""


Всего размечено 86 тем. Из таблицы можно заметить, что есть темы, которые пересекаются, например "О войне", "военный", "фронтовый", "тыл".

In [5]:
tags.shape

(86, 3)

In [6]:
# список тем типа 1
themes = list(tags['tag'])

In [7]:
diaries = pd.read_csv('data/diary.csv', sep=',#', header=None)
diaries.head()

,0,1,2,3,4,5,6,7,8,9,10
0,"""3""","""4""","""published""","""329""","""1932-09-28""","""1937-04-30""",NaN,"""0""","""0""","""0""","""1425446154"""
1,"""4""","""5""","""published""","""356""","""1942-09-09""","""1944-01-27""",NaN,"""0""","""0""","""1474033861""","""1425446228"""
2,"""5""","""6""","""published""","""28""","""1943-01-31""","""1951-07-03""",NaN,"""0""","""0""","""0""","""1425446370"""
3,"""6""","""7""","""published""","""65""","""0000-00-00""","""0000-00-00""",NaN,"""0""","""0""","""0""","""1425446645"""
4,"""8""","""11""","""published""","""804""","""1943-08-04""","""1961-12-31""",NaN,"""0""","""0""","""0""","""1425487703"""


Всего есть 2342 дневника.

In [8]:
len(diaries[0])

2342

In [9]:
tags_diaries = pd.read_csv('data/tags_diaries.csv', sep=',#', names=['diary', 'tag'])
tags_diaries.head()

,diary,tag
0,"""3""","""181"""
1,"""3""","""328"""
2,"""4""","""180"""
3,"""4""","""184"""
4,"""8""","""187"""


Из них хоть какой-то тэг приписан к 1407 дневникам. К одному дневнику может быть приписано несколько тэгов.

In [10]:
len(set(tags_diaries['diary']))

1407

In [11]:
# датафрейм с id дневников, которым приписана хотя бы одна тема
diaries_with_themes = tags_diaries[tags_diaries['tag'].isin(themes)]

Тема приписана 1407 дневникам. Многим дневникам приписано несколько тем. Видимо, потому, что некоторое темы пересекаются или у одного дневника может быть несколько разных тем. 

Как можно заметить, локации и персоны ни одному дневнику не приписаны. 

In [12]:
len(set(diaries_with_themes['diary']))

1407

In [13]:
with open('data/notes.json', 'r') as f:
        notes = json.load(f)

В джейсоне 182975 записи (кажется, в csv было больше).

In [14]:
len(notes[2]['data'])

182975

In [26]:
notes[2]['data'][-1]

{'createdDate': '1485536357',
 'date': '2002-03-18',
 'dateTop': '0000-00-00',
 'diary': '145',
 'id': '221877',
 'julian_calendar': '0',
 'notDated': '0',
 'pictures': '1',
 'text': 'Впервые в жизни получил какое-то удовольствие от тренажера. Нынче он у нас стал компьютерным, цветным, адекватным. Не все, конечно, еще отлажено, но против старого — небо и земля. На нем можно решать задачи. С пожаром на взлете мы приземлились за 2.45, визуально, ну, с чуть взмокшей спиной. Так это ж я, старик; молодые-то быстрее зайдут.    \nПриехала газетчица Маша; дождался, посмотрел материал. От всего разговора, в статью вошли, конечно, крохи, и не самые лучшие; пришлось править, кое-что даже принципиально. Для газетенки сойдет. Сказала, читайте послезавтра в газете… Кофейком ее напоил… но все как-то на скоростях. Оно и понятно: репортеры — они как хариусы: хвать на лету — и дальше…    \nЧем мне, собственно, не понравился материал? Тем, что в этом интервью о моей большой книге — ни слова. Вообще, на м

In [15]:
rows = []

for note in notes[2]['data']:
    rows.append([note['text'], note['diary']])

In [16]:
notes = pd.DataFrame(rows, columns=['text', 'diary'])

In [17]:
# датафрейм с записями и id их дневника
notes.head()

,text,diary
0,Барабинск. Татарка.,130
1,Омск. Утро. Стоим до вечера. Обед. Сухой паек.,130
2,В 6 часов утра еще темно. Станция Называевская...,130
3,Утром рано приехали в Тюкалинск. Из автоотряда...,130
4,"Так давно я не прикасался к дневнику, что он м...",234


# Предобработка

In [18]:
from stop_words import get_stop_words
import nltk
from pymystem3 import Mystem
import re

In [19]:
random_seed = 42

In [20]:
stop_words = get_stop_words('ru')
stop_words.append('свой')

In [21]:
m = Mystem()

In [22]:
def tokenize(text):
    comp = re.compile('[А-ЯЁа-яё\-]+')
    tokens = comp.findall(text.lower())
    return tokens

In [23]:
def lemmatize(text):
    return [m.lemmatize(word)[0] for word in text]

In [25]:
# токенизируем заметки
notes_lemma = pd.DataFrame()
notes_lemma['text'] = notes.text[:10000].apply(lambda x: tokenize(x))

In [26]:
# лемматизируем заметки
notes_lemma['text'] = notes_lemma.text.apply(lambda x: lemmatize(x))

In [32]:
notes_lemma = notes_lemma.text.apply(lambda x: ' '.join(x))

In [33]:
notes_lemma.head(10)

0                                    барабинск татарка
1            омск утро стоять до вечер обед сухой паек
2    в час утро еще темно станция называевский слез...
3    утро рано приезжать в тюкалинск из автоотряд и...
4    так давно я не прикасаться к дневник что он я ...
5    еще проходить пешком более км навстречу выходи...
6    стоять на место за это время поступать несколь...
7    приезжать на станция потом идти до военный гор...
8    мы отделять от рот в отдельный комната вчера б...
9    мы перевозить в расположение бригада где сформ...
Name: text, dtype: object

# Тематическое моделирование

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import FeatureUnion

In [25]:
tfidf = TfidfVectorizer(sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(1, 2))

In [35]:
notes_vec = tfidf.fit_transform(notes_lemma)

In [26]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [37]:
tfidf_feature_names = tfidf.get_feature_names()

In [38]:
no_topics = 10

# NMF
nmf = NMF(n_components=no_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(notes_vec)

In [27]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [45]:
no_top_words = 15
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
которыи говорить какои час идти дело такои давать вечер немец фронт получать сеичас другои новыи
Topic 1:
мао кпк цзэ мао цзэ цк кпк цк председатель цк китаи председатель чан каи чан каи ван кан руководство
Topic 2:
язык болгарскии славянскии статья славянскии язык институт болгарскии язык славяноведение сравнительныи языкознание доклад институт славяноведение атлас словарь печать
Topic 3:
паровоз подорвать вагон эшелон разбивать разбивать паровоз подорвать поезд повреждать балицкии паровоз вагон поезд раион вагон повреждать паровоз разбивать вражескии эшелон
Topic 4:
тов отряд федоров елезныи елезныи д тов федоров рота группа д орога орога командир партизан д соединение посылать
Topic 5:
миша пьеса театр дон кихот кихот эрдман вечер оля дон звонок мхат дмитриев позвонить борис приити
Topic 6:
японец снаряд батарея японскии гора гавань стрелять фланг форт позиция огонь час минута генерал стессель обстреливать
Topic 7:
егоров вечер богоявленскии н петр утро н егоров д н прогулк

In [41]:
# LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(notes_vec)

/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [46]:
display_topics(lda, tfidf_feature_names, no_top_words)

Topic 0:
миноискатель находка миноискатель находка обед стоять особенныи отправлять курсы леитенант праздничныи исключение блюдо обед особенныи праздничныи исключение улучшать улучшать блюдо дорогобуж можаиск дорогобуж можаиск ночь стоять можаиск
Topic 1:
граница монголия эвакуация тыл большии эвакуация канск обед час канск канск борович баня борович эвакуация борович баня перевязка первыи ремонт дорога продукт балезино обед плохои стоять ст балезино
Topic 2:
барабинск ответ жена эвакуация борович борович ответ тася часть продукт кот тася стоять парк квартира стоять письмо кот перово станция зима зима вечер час верх томочка
Topic 3:
вечер которыи говорить час утро идти такои давать какои работа вчера дело получать новыи письмо
Topic 4:
обыкновенныи госпиталь смертень месяц голодень рассказывать прибавляться голодень прибавляться месяц голодень пухня сухня пухня сухня сухня смертень пухня ожидать эшелон автор дневник проезжать курган мастерская зашивка
Topic 5:
ачинск красноярск ачинск 

In [48]:
def get_topics(texts):
    
    notes_lemma = pd.DataFrame()
    notes_lemma['text'] = texts.apply(lambda x: tokenize(x))
    notes_lemma['text'] = notes_lemma.text.apply(lambda x: lemmatize(x))
    notes_lemma = notes_lemma.text.apply(lambda x: ' '.join(x))
    
    notes_vec = tfidf.fit_transform(notes_lemma)
    tfidf_feature_names = tfidf.get_feature_names()
    
    no_topics = 10
    nmf = NMF(n_components=no_topics, random_state=random_seed, alpha=.1, 
              l1_ratio=.5, init='nndsvd').fit(notes_vec)
    
    no_top_words = 20
    display_topics(nmf, tfidf_feature_names, no_top_words)

In [48]:
get_topics(notes.text[10000:20000])

Topic 0:
которыи какои такои говорить другои знать становиться идти дело давать жить воина самыи сеичас час
Topic 1:
утро арт ч пальба ч утро погода слышныи стрельба город редкии выстрел арт пальба ночь облачность тихо
Topic 2:
ч мина мина ч тревога отбои зенитка тревога ч воздушныи тревога мина ч воздушныи отбои ч отбои давать возд тревога возд давать ч
Topic 3:
спектакль играть роль театр отелло актер ю сцена зал пьеса маскарад акт хорошии зритель искусство
Topic 4:
временныи временныи правительство правительство петроград россия конрад конотоп херсон конотоп урюпинскии конрад промелькнуть конотоп вправо конотоп старыи конотоп попадать конотоп одновременно конотоп немец конрад россия
Topic 5:
снаряд обстрел город раион разрыв обстрел город враг свист мпво штаб мпво рваться ч радио ул слышныи
Topic 6:
взвод батальон гаубица красныи батарея полк орудие полковник команда пехота позиция огонь колонна открывать огонь командир
Topic 7:
арт обстрел обстрел арт начинаться арт раион ч одиночн

In [29]:
get_topics(notes.text[-10000:])

Topic 0:
которыи говорить такои какои дело вечер час другои идти становиться знать вопрос давать работа сеичас
Topic 1:
снег выпадать выпадать снег тепло снег снег растаивать растаивать мороз снег река подмораживать оттепель санныи санныи путь морозец погода обильныи снег
Topic 2:
л н л н софья андреевна андреевна софья львовна ясныи александра львовна нынче александра письмо чертков играть шахматы
Topic 3:
улеи пчела мед выставлять матка погода пчела работать выставлять улеи пуд фунт убирать магазин рои пчела улеи роить
Topic 4:
тепло тепло снег дождь тепло становиться тепло тепло градус снежок тепло становиться становиться тепло дождь погода тепло таять растаивать тепло снежок тепло тихо снег растаивать
Topic 5:
скорость посадка полет экипаж летать пилот садиться полоса машина самолет заход высота метр глиссада реис
Topic 6:
церковь митрополит служить дарить народ воскресение владыка служба народ церковь литургия молебен храм которыи архиепископ николаи
Topic 7:
мороз мороз снег моро

# Записи без тематических тэгов

У 104184 записи нет тематического тэга.

In [45]:
notes_without_themes = notes[~notes['diary'].isin(diaries_with_themes['diary'].apply(lambda x: x.strip('"')))]

In [46]:
notes_without_themes.shape

(104184, 2)

In [49]:
get_topics(notes_without_themes.text[:10000])

Topic 0:
которыи говорить какои такои дело давать идти другои москва вечер знать вчера становиться час русскии новыи писать приходить утро сеичас
Topic 1:
аля час обед веранда алена копель завтрак фира вечер сидеть копа дома море али солнце записывать четверг катя спать небо
Topic 2:
спектакль играть театр роль отелло актер ю сцена пьеса зал маскарад акт репетиция хорошии зритель ленинградскии проспект смотреть первыи сила искусство
Topic 3:
егоров н егоров н д н богоявленскии д готье вечер университет утро любавскии воскресение петр л иза книга историческии иза веселовскии герье заседание
Topic 4:
сценарии писать сценарии писать сценарии плохо плохо сценарии дело комитет позвонить комитет радио комитет расстреливать комитет расставаться комитет рассматривать комитет рассказывать комитет раскалывать комитет разрешать комитет разоружать комитет разныи комитет разговор комитет радиовещание комитет работать комитет рабочии
Topic 5:
роман работа роман свободныи роман роман вечер вечер чита

In [50]:
get_topics(notes_without_themes.text[-10000:])

Topic 0:
которыи говорить какои такои знать вечер другои думать становиться работа час идти хорошии дело сеичас давать самыи приходить писать оставаться
Topic 1:
текущии текущии работа бсэ работа работа бсэ пролетариат библиография история пролетариат библиографическии редактировать библиографическии комиссия история библиография история комиссия карточка работа библиографическии статья редактировать библиография проредактировать бсэ текущии
Topic 2:
ч доклад ученыи ученыи совет институт заседание совет история ч ученыи аон защита группа ч заседание наука революционныи ситуация историческии наука историческии собрание революционныи история историческии
Topic 3:
снег мороз мороз снег выпадать выпадать снег погода снег растаивать тепло снег морозец растаивать река градус оттепель стоять подмораживать мороз градус санныи санныи путь вьюга зима
Topic 4:
николина николина горе горе николина гора гора горе работать уезжать николина дод уезжать отъезд николина учебник класс работа николина уч